In [1]:
import pandas as pd
import numpy as np

from IPython.display import clear_output

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import pdb

In [2]:
text_reference_sampled_df = pd.read_csv('text_reference_sampled_df.csv')
text_reference_sampled_df.head()

,reference_id,reference_property_id,reference_datatype,url,netloc,netloc_agg,error_msg,code,content-type,final_url,reason,language_crawl,language_crawl_score,sampling_weight_vb,sampling_weight
0,390d6c6e68a32e11f8d7b0883cda0557db529fe6,P4656,url,https://en.wikipedia.org/w/index.php?title=G._...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=G._...,OK,en,0.934613,321744|13,24749.538462
1,49b9aec8e10815611ff0379a34d5fd7c3830566e,P4656,url,https://en.wikipedia.org/w/index.php?title=Seb...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Seb...,OK,en,0.650086,321744|13,24749.538462
2,ab3e9ada7246257ffbfb86fa90a54f25e45a704e,P4656,url,https://en.wikipedia.org/w/index.php?title=Ave...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Ave...,OK,en,0.842498,321744|13,24749.538462
3,76b04346ad57869d9e5ae1007ba8343d708ab6f9,P4656,url,https://en.wikipedia.org/w/index.php?title=Yut...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Yut...,OK,en,0.608352,321744|13,24749.538462
4,1ee71b39caf6df395c64b436fea4895692812d38,P4656,url,https://en.wikipedia.org/w/index.php?title=Dol...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Dol...,OK,en,0.856581,321744|13,24749.538462


# Extract HTML

For each url in the sampled dataset, we will:
1. Visit the URL with a *headed* browser
2. Wait for the correct content to be rendered
3. Extract the html code and save it into a folder, but also in the dataframe

In [3]:
_RE_COMBINE_WHITESPACE = re.compile(r"\s+")

options = Options()
#options.add_argument("--headless")
options.add_argument("--incognito")
#options.add_argument("start-maximized")
#options.add_argument("--disable-extensions")
#options.add_argument("--enable-javascript")
#options.add_argument("--window-size=1920x1080")
#options.add_experimental_option("excludeSwitches", ["enable-automation"])
#options.add_experimental_option('useAutomationExtension', False)
#options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36")

DRIVER_PATH = 'chromedriver.exe'
ser = Service(DRIVER_PATH)

text_reference_sampled_df['html'] = None

In [21]:
from os.path import exists
import os
from pathlib import Path

# Driver outside so we keep a single browser and history
# NOTE: Some URLs lead to PDF linked by 'embed' tags. Pursue those and try processing them with an adequate tools.
with webdriver.Chrome(options=options, service=ser) as driver:
    
    driver.set_window_position(-1500, 0)
    driver.maximize_window()
    for i, row in text_reference_sampled_df.iterrows():
        
        if not exists(Path('reference_html') / row.netloc):
            os.makedirs(Path('reference_html') / row.netloc)
        if exists(Path('reference_html') / row.netloc / row.reference_id):
            if text_reference_sampled_df.loc[i,'html'] is None:
                print(i,' - Loading from disk - ', row.url)
                with open(Path('reference_html') / row.netloc / row.reference_id, 'r', encoding='utf-8') as f:
                    text_reference_sampled_df.loc[i,'html'] = f.read()
            continue      
            
        print(i, row.url)
        driver.get(row.url)
        pdb.set_trace()
        html = driver.page_source
        
        with open(Path('reference_html') / row.netloc / row.reference_id, 'w+', encoding='utf-8') as f:
            f.write(html)
            
        text_reference_sampled_df.loc[i,'html'] = html
        
        clear_output(wait=True)
clear_output(wait=True)

``p driver.switch_to.frame('ext-gen66')
p BeautifulSoup(driver.page_source, "lxml").get_text(' ')``

**NOTE**: Some URLs lead to PDF linked by 'embed' tags. Pursue those and try processing them with an adequate tools.
    
    - https://www.bailii.org/(.*).html change html to pdf and process that instead if 'This item of legislation is only available to download and view as PDF' included in the html body.
    - https://www.eib.org/en/ also links to lots of pdfs
    
**NOTE**: Some content is not in English, like the following, which should be either removed or replaced.

    - https://www.archinform.net/projekte/19632.htm (290)
    - https://www.archinform.net/projekte/11996.htm (295)
    
Since it's 4 out of 676, let's just erase them.

In [25]:
#html = text_reference_sampled_df.html[42]
#
#
#soup = BeautifulSoup(html, "lxml")
#[s.decompose() for s in soup("script")]  # remove <script> elements
#body_text = _RE_COMBINE_WHITESPACE.sub(" ", soup.body.get_text(' ')).strip()
#body_text
#

In [ ]:
text_reference_sampled_df.to_csv('text_reference_sampled_df_html.csv', index=None)